This notebook demos how to make prompt functions -- that is, python functions that 
are defined by prompt templates.

# Pre-defined prompt functions

We have a few pre-defined prompt functions that come with the `oa` package. 
We'll start looking at those, and then see how to make them...

In [1]:
from oa import ask
from inspect import signature

print_signature = lambda func: print(signature(func))

list(ask.ai)[:10]

['prompt_template_improver',
 'user_story_to_code_for_dag',
 'prompt_template_starter',
 'make_synopsis',
 'description_to_software_specs',
 'define_jargon',
 'simple_tests_for_code',
 'suggest_names',
 'specs_to_code_for_dag']

## prompt_template_starter

Of particular interest to use here is an AI-enabled function that will help us make AI-enabled functions...

It has two inputs:
- a `task`, which is a specification of what we want a prompt template for
- `inputs`, which is a specification of what "variables" this prompt template should have, that is, what kinds of inputs we're going to ask the user

In [13]:
print_signature(ask.ai.prompt_template_starter)

(task, *, inputs=' ')


First we go through a bit of experimentation, trying out different ways of expressing what we want. 

In [14]:
template = ask.ai.prompt_template_starter(
    task="""
    Help me learn a new language
    """,
    inputs = ""
)
print(template)

You are a language teacher. Teach me how to say {this} in the new language.


In [20]:
template = ask.ai.prompt_template_starter(
    task="""
    Help me learn new vocabulary words of a language that I'm learning
    by giving me some sentences that I should translate into the language I am learning.
    """,
    inputs = "language, vocabulary_words, number_of_sentences"
)
print(template)

You are a language learning expert. Translate the following sentences into {language} using the vocabulary words: {vocabulary_words}. Translate {number_of_sentences} sentences.


Then we settle on some template, possibly editing it a bit, and make a prompt function out of it.

In [22]:
from oa import prompt_function

template = """
You are a language learning expert. Translate the following sentences into {language:Japanese} 
using the vocabulary words: {vocabulary_words}. 
Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where 
you'll list the actual translations.
"""
practice_vocab = prompt_function(template)
print_signature(practice_vocab)

(vocabulary_words, *, language='Japanese', number_of_sentences='3')


Let's try this function out:

In [23]:
print(practice_vocab("tasty, sushi, hello, please, thank you"))

1. Hello, please give me some tasty sushi.
2. Thank you for the tasty sushi.
3. Hello, may I please have some more tasty sushi?

ANSWERS:
1. こんにちは、美味しい寿司をください。
2. 美味しい寿司をありがとうございます。
3. こんにちは、美味しい寿司をもう少しいただけますか。


Let's have a look at what the underlying template looks like.

In [24]:
print(ask.ai.prompt_template_starter.template_original)


```
You are an expert prompt engineer.
I will give you a task below and would like you to write a “prompt template” to perform this task.
This task will be parametrized by some inputs, which should appear in the prompt template as placeholders, marked by braces {like} {this} (here, both “like” and “this” are names of inputs that would parametrize the prompt template). 
I might give you an explicit list of inputs, which will be a comma or space separated list of strings (to be more precise, python identifier strings (so only alphanumerics and underscores).
If I do give you this list of inputs (names), you should use them (all) in the prompt template, using the {braces} to show where these should be injected when making a prompt (I have code for that). 
You should use them all.
If I don’t give you this list of inputs (that is, my list of inputs is empty), you should come up with your own. Remember; I’m going to use this template to make a form to get inputs from the user, so I need this

## prompt_template_improver

So now we have a template that works. How can we improve it?

Let's use another (prompt-based) function for that

In [2]:
template = """
You are a language learning expert. Translate the following sentences into {language:Japanese} 
using the vocabulary words: {vocabulary_words}. 
Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where 
you'll list the actual translations.
"""

In [3]:
print_signature(ask.ai.prompt_template_improver)

(prompt_template, *, specific_goals=' ', number_of_alternatives='3', include_additional_explanations='false', prompt_engineering_tips='false')


In [5]:
alternative_template_1 = ask.ai.prompt_template_improver(template)
print(alternative_template_1)

**Prompt Alternatives:**
1. You are a language learning expert. Translate the following sentences into {language:Japanese} using the vocabulary words: {vocabulary_words}. Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where you'll list the actual translations.
  
2. As a language learning expert, I need you to translate the following sentences into {language:Japanese} with the provided vocabulary words: {vocabulary_words}. Provide a list of {number_of_sentences:3} to translate, followed by an ANSWERS section where the actual translations will be listed.

3. Calling all language learning experts! Your task is to translate the following sentences into {language:Japanese} using the vocabulary words: {vocabulary_words}. I need a list of {number_of_sentences:3} translations, followed by an ANSWERS section detailing the actual translations.




In [6]:
alternative_template_2 = ask.ai.prompt_template_improver(
    template,
    specific_goals='To stay more focused on the vocabulary words, departing from them as little as possible', 
    number_of_alternatives=2, 
    include_additional_explanations=True, 
    prompt_engineering_tips=True
)
print(alternative_template_2)

Alternative 1:
You are a language learning expert. Translate the following sentences into {language:Japanese} 
using the vocabulary words: {vocabulary_words}. 
Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where 
you'll list the actual translations.

Alternative 2:
As a language learning expert, translate the sentences below into {language:Japanese} 
with the given vocabulary words: {vocabulary_words}. 
Provide {number_of_sentences:3} translations and list them in the ANSWERS section.

ADDITIONAL EXPLANATIONS:
In these alternatives, the focus is on the vocabulary words provided, ensuring that the translations stick closely to them. By structuring the prompt in a specific format, it can help guide the expert towards using the vocabulary effectively in the translations.

PROMPT ENGINEERING TIPS:
1. Encourage the expert to utilize the vocabulary words creatively by providing a variety of sentence structures to translate.
2. Remind the expert to pa

## changing the chat model

Not good enough?

Perhaps the prompt is good enough, but not the ai?

Let's try to change the model we're using!

In [11]:
from oa.base import list_engine_ids

list_engine_ids()


['whisper-1',
 'davinci-002',
 'dall-e-2',
 'tts-1-hd-1106',
 'tts-1-hd',
 'gpt-3.5-turbo',
 'gpt-3.5-turbo-0125',
 'gpt-3.5-turbo-instruct-0914',
 'gpt-3.5-turbo-16k-0613',
 'gpt-3.5-turbo-16k',
 'gpt-3.5-turbo-instruct',
 'gpt-3.5-turbo-0301',
 'gpt-3.5-turbo-0613',
 'tts-1',
 'dall-e-3',
 'gpt-3.5-turbo-1106',
 'babbage-002',
 'gpt-4-0125-preview',
 'gpt-4-turbo-preview',
 'tts-1-1106',
 'text-embedding-3-large',
 'gpt-4-turbo-2024-04-09',
 'gpt-4-vision-preview',
 'text-embedding-3-small',
 'gpt-4',
 'text-embedding-ada-002',
 'gpt-4-1106-vision-preview',
 'gpt-4-1106-preview',
 'gpt-4-0613',
 'gpt-4-turbo']

In [15]:
from oa import prompt_function, chat
from functools import partial

smarter_chat = partial(chat, model='gpt-4')
smarter_prompt_function = partial(prompt_function, prompt_func=smarter_chat)
# and now let's make a smarter prompt_template_improver
# Since prompt_template_improver was already made with the default chat model,
# we need to go back to the template, and make a new function with the new model
smarter_prompt_template_improver = smarter_prompt_function(
    ask.ai.prompt_template_improver.template_original
)

In [17]:
template = """
You are a language learning expert. Translate the following sentences into {language:Japanese} 
using the vocabulary words: {vocabulary_words}. 
Give me a list of {number_of_sentences:3} to translate followed with an ANSWERS section where 
you'll list the actual translations.
"""
alternative_template_3 = smarter_prompt_template_improver(
    template,
    specific_goals='To stay more focused on the vocabulary words, departing from them as little as possible', 
    number_of_alternatives=2, 
    include_additional_explanations=True, 
    prompt_engineering_tips=True
)
print(alternative_template_3)

Alternative 1:
For our language mastery exercise today, you are to translate the following phrases into {language:Japanese}. You will be utilizing these key phrases: {vocabulary_words}. Provide an index of {number_of_sentences:3} notions to translate and follow it with a SOLUTIONS segment where you catalog the accurate translations.

Alternative 2:
You are recognized for your proficiency in languages. The task here requires you to translate the subsequent sentences into {language:Japanese}, with a focus on these keywords: {vocabulary_words}. Try to construct and translate {number_of_sentences:3} original lines, followed by an ANSWERS segment displaying the correct translations.

ADDITIONAL EXPLANATIONS:

Alternative 1 slightly modifies the tone and language of the original template. In this alternative, the wording is a bit more formal. 'mastery exercise' is used instead of 'learning', which might help the AI take a more scholarly approach to the task. Also, by renaming the 'ANSWERS' t

# Your own templates

I mean, really we've already gone through the motions now, perhaps unknowlingly.

We took a pre existing ai-based function as our point of departure, but have seen how it itself was 
made from a prompt template string. 

##